In [ ]:
from pydicom import dcmread
from os import listdir
from os.path import isfile, join, isdir
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# change file path for appropriate one
base_path = '/home/ADNI/'
csv_path = '/home/ADNI/3T_3D_1mm_4_05_2020.csv'

save_path = '/home/basimova_nf/ADNI-processed/images/'
csv_save_path = '/home/basimova_nf/ADNI-processed/data.csv'

In [ ]:
all_file_names = []

In [ ]:
# based on data structure
for folder1 in listdir(base_path):
    if isdir(f'{base_path}{folder1}'):
        for folder2 in listdir(f'{base_path}{folder1}'):
            for folder3 in listdir(f'{base_path}{folder1}/{folder2}'):
                for folder4 in listdir(f'{base_path}{folder1}/{folder2}/{folder3}'):
                    one_picture_files = []
                    for file_name in listdir(f'{base_path}{folder1}/{folder2}/{folder3}/{folder4}'):
                        one_picture_files.append(f'{base_path}{folder1}/{folder2}/{folder3}/{folder4}/{file_name}')
                    all_file_names.append(one_picture_files)

In [ ]:
sorted_files = []
for i, one_picture_files in enumerate(all_file_names):
    sorted_files.append(sorted(one_picture_files, key=lambda string: int(string.split('_')[-3])))

In [ ]:
data_csv = pd.read_csv(csv_path)
# data_csv.drop(columns=['Unnamed: 0'], inplace=True)
data_csv['File Names'] = data_csv['Group']

In [ ]:
def get_file_id(file_name):
    ID = file_name.split('_')[-1][1:-4]
    return int(ID)
def make_good_file_name(full_file_name):
    return '-'.join(full_file_name.split('/')[4:8])

In [ ]:
bad_files_id = set()
bad_folders = []

In [ ]:
# will not use bad small pictures
min_images_number = 174

In [ ]:
for one_folder in sorted_files:
    print('____')
    try:
        if len(one_folder) >= min_images_number:
            ID = get_file_id(one_folder[0])
            base_shape = dcmread(one_folder[0]).pixel_array.shape
            img = np.zeros((len(one_folder), *base_shape))
            for i, file in enumerate(one_folder):
                print(i/len(one_folder))
                dcm_file = dcmread(file)
                img[i] = dcm_file.pixel_array
            np.save(save_path+make_good_file_name(file), img)
            data_csv.loc[data_csv['Image Data ID']==ID, 'File Names'] = make_good_file_name(file)
        else:
            bad_folders.append(one_folder)
            if len(one_folder) > 0: bad_files_id.add(get_file_id(one_folder[0]))
    except:
        bad_folders.append(one_folder)
        bad_files_id.add(get_file_id(file))

In [ ]:
data_csv = data_csv[~data_csv['Image Data ID'].isin(bad_files_id)]

In [ ]:
data_csv.to_csv(csv_save_path)

In [ ]:
plt.imshow(img[img.shape[0]//2])

In [ ]:
img.shape